In [1]:
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [2]:
df =read_csv('pong.csv', sep=r'\s*,\s*')

/home/jatin/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
print(df.shape)

(12232, 6)


In [41]:
print(df.describe())

           ball_cx      ball_cy      ball_vx      ball_vy     paddle_y  \
count  6104.000000  6104.000000  6104.000000  6104.000000  6104.000000   
mean    398.835026   300.525721     0.478702     0.183814   245.588467   
std     220.794219   168.790775     2.961804     2.994609   131.343241   
min       7.000000     7.000000    -3.000000    -3.000000    -2.000000   
25%     208.000000   154.000000    -3.000000    -3.000000   157.000000   
50%     399.000000   298.000000     3.000000     3.000000   265.000000   
75%     589.500000   448.000000     3.000000     3.000000   343.000000   
max     793.000000   592.000000     3.000000     3.000000   478.000000   

         paddle_vy  
count  6104.000000  
mean      0.089941  
std       1.903696  
min      -3.000000  
25%       0.000000  
50%       0.000000  
75%       0.000000  
max       3.000000  


In [5]:
df

,ball_cx,ball_cy,ball_vx,ball_vy,paddle_y,paddle_vy
0,63,103,3,3,10,0
1,66,106,3,3,10,0
2,69,109,3,3,10,0
3,72,112,3,3,10,0
4,75,115,3,3,10,0
...,...,...,...,...,...,...
9422,781,289,3,3,211,-3
9423,784,292,3,3,208,-3
9424,787,295,3,3,205,-3
9425,790,298,3,3,202,-3


In [6]:
print(df.groupby('paddle_vy').size())

paddle_vy
-3    1565
 0    6111
 3    1751
dtype: int64


In [7]:
# Split-out validation dataset
array = df.values
X = array[:,0:5]
y = array[:,5]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

In [49]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: 0.597584 (0.000828)
LDA: 0.597584 (0.000828)
KNN: 0.855011 (0.012747)
CART: 0.890233 (0.010789)
NB: 0.588160 (0.012141)
SVM: 0.597993 (0.002865)


In [50]:
# Make predictions on validation dataset
model = SVC(gamma='auto')
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)

In [51]:
# Evaluate predictions
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.5954135954135954
[[  7 218   0]
 [  2 720   1]
 [  0 273   0]]
              precision    recall  f1-score   support

          -3       0.78      0.03      0.06       225
           0       0.59      1.00      0.74       723
           3       0.00      0.00      0.00       273

    accuracy                           0.60      1221
   macro avg       0.46      0.34      0.27      1221
weighted avg       0.50      0.60      0.45      1221

